In [19]:
import pandas as pd 
import numpy as np
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
df = pd.read_csv("tmdb_movies_data.csv").reset_index()
df


,index,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,1.379999e+08,1.392446e+09
1,1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,1.379999e+08,3.481613e+08
2,2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,1.012000e+08,2.716190e+08
3,3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,1.839999e+08,1.902723e+09
4,4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,10861,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,...,"The Endless Summer, by Bruce Brown, is one of ...",95,Documentary,Bruce Brown Films,6/15/1966,11,7.4,1966,0.000000e+00,0.000000e+00
10862,10862,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,...,Grand Prix driver Pete Aron is fired by his te...,176,Action|Adventure|Drama,Cherokee Productions|Joel Productions|Douglas ...,12/21/1966,20,5.7,1966,0.000000e+00,0.000000e+00
10863,10863,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,...,An insurance agent who moonlights as a carthie...,94,Mystery|Comedy,Mosfilm,1/1/1966,11,6.5,1966,0.000000e+00,0.000000e+00
10864,10864,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,...,"In comic Woody Allen's film debut, he took the...",80,Action|Comedy,Benedict Pictures Corp.,11/2/1966,22,5.4,1966,0.000000e+00,0.000000e+00


In [21]:
features = ['keywords', 'cast', 'genres', 'director', 'overview', 'production_companies', 'tagline']
for feature in features:
    df[feature] = df[feature].fillna('')

def combined_features(row):
    return row['keywords'].replace('|', " ")+" "+row['cast'].replace('|', " ")+" "+row['genres'].replace('|', " ")+row['genres'].replace('|', " ")+row['genres'].replace('|', " ")+row['genres'].replace('|', " ")+" "+row['director']+" "+row['overview']+" "+row['production_companies'].replace('|', " ")+" "+row['tagline']
df["combined_features"] = df.apply(combined_features, axis =1)
df["combined_features"]

0        monster dna tyrannosaurus rex velociraptor isl...
1        future chase post-apocalyptic dystopia austral...
2        based on novel revolution dystopia sequel dyst...
3        android spaceship jedi space opera 3d Harrison...
4        car race speed revenge suspense car Vin Diesel...
                               ...                        
10861    surfer surfboard surfing Michael Hynson Robert...
10862    car race racing formula 1 James Garner Eva Mar...
10863    car trolley stealing car Innokentiy Smoktunovs...
10864    spoof Tatsuya Mihashi Akiko Wakabayashi Mie Ha...
10865    fire gun drive sacrifice flashlight Harold P. ...
Name: combined_features, Length: 10866, dtype: object

In [22]:
print(" original dataframe \n", df["combined_features"])
 
# replace '_' with '-'
#df["combined_features"] = df["combined_features"].replace('|', " ")
 
#print dataframe
#print(" After replace character \n", df["combined_features"])

 original dataframe 
 0        monster dna tyrannosaurus rex velociraptor isl...
1        future chase post-apocalyptic dystopia austral...
2        based on novel revolution dystopia sequel dyst...
3        android spaceship jedi space opera 3d Harrison...
4        car race speed revenge suspense car Vin Diesel...
                               ...                        
10861    surfer surfboard surfing Michael Hynson Robert...
10862    car race racing formula 1 James Garner Eva Mar...
10863    car trolley stealing car Innokentiy Smoktunovs...
10864    spoof Tatsuya Mihashi Akiko Wakabayashi Mie Ha...
10865    fire gun drive sacrifice flashlight Harold P. ...
Name: combined_features, Length: 10866, dtype: object


In [23]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
#print("Count Matrix:", count_matrix.toarray())

cosine_sim = cosine_similarity(count_matrix)

In [24]:
movie_user_likes = "Star Wars"
if (movie_user_likes in df['original_title'].unique()) == False:
    raise Exception("movie not found")

def get_index_from_title(title):
    return df[df["original_title"] == title]["index"].values[0]
movie_index = get_index_from_title(movie_user_likes)

similar_movies = list(enumerate(cosine_sim[movie_index]))
#similar_movies

sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)



def get_title_from_index(index):
    return df[df.index == index]["original_title"].values[0]

sorted_by_popularity = []

i=0
for movie in sorted_similar_movies:
    #print(get_title_from_index(movie[0]))
    #print(sorted_similar_movies[i])
    sorted_by_popularity.append({"TITLE":get_title_from_index(movie[0]),"Popularity":df[df.index == movie[0]]["popularity"]})
    #print(df[df.original_title == "Jurassic World"]["popularity"])
    i=i+1
    if i>50:
        break

df_sorted_by_popularity = pd.DataFrame(sorted_by_popularity)
df_sorted_by_popularity


,TITLE,Popularity
0,Star Wars,"1329 12.037933 Name: popularity, dtype: flo..."
1,The Empire Strikes Back,"7309 5.488441 Name: popularity, dtype: float64"
2,Return of the Jedi,"7987 4.828854 Name: popularity, dtype: float64"
3,Battle For SkyArk,"286 0.437088 Name: popularity, dtype: float64"
4,Family Guy Presents: It's a Trap!,"2172 0.374797 Name: popularity, dtype: float64"
5,G.I. Joe: The Rise of Cobra,"1407 1.906998 Name: popularity, dtype: float64"
6,Fallen: The Destiny,"7809 0.060795 Name: popularity, dtype: float64"
7,Airwolf: The Movie,"7973 0.154225 Name: popularity, dtype: float64"
8,Fantastic Voyage,"10848 0.207257 Name: popularity, dtype: flo..."
9,The Chronicles of Riddick,"7024 1.052176 Name: popularity, dtype: float64"
